# 전이학습 모델 사용

# 1. 중요 라이브러리 임포트

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import torchvision
from torchvision import transforms
from torchvision.datasets import FashionMNIST, ImageFolder
from torchvision.models import vgg16

import numpy as np
from sklearn.metrics import accuracy_score

# 2. DataSet 준비

In [ ]:
transform = transforms.Compose([transforms.Resize((256, 256)),
                                transforms.RandomHorizontalFlip(0.5),
                                transforms.ToTensor()])

train_set = ImageFolder("D:/0_Data/Cat_Dog/training_set/", transform =transform)
test_set = ImageFolder("D:/0_Data/Cat_Dog/test_set/", transform=transform)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=True)

In [ ]:
model = vgg16()
model

# 3. 모델 클래스 생성

In [ ]:
class custom_ResNet(nn.Module):
    def __init__(self):
        super(custom_ResNet, self).__init__()
        self.vgg_layer = vgg16(weights=None) # 가중치를 가져오지 않음
        self.fc0 = nn.Linear(1000, 256) # VGG16의 최종의 출력을 1000에서 256으로 바꿈
        self.fc1 = nn.Linear(256, 32)
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        x = self.vgg_layer(x)
        x = self.fc0(x)
        x = F.relu(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

In [ ]:
model = custom_ResNet()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
def train(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data in (dataloader):
        inputs, targs = data

        optimizer.zero_grad()

        y_pred = model(inputs)
        loss = criterion(y_pred, targs)
        total_loss += loss

        loss.backward()
        optimizer.step()
    
    return total_loss / len(dataloader)


def eval(model, dataloader, criterion):
    model.eval()

    y_pred_lst = []
    y_targ_lst = []
    loss = 0
    for idx, data in (enumerate(dataloader)):
        inputs, targs = data

        y_pred = model(inputs)

        loss += criterion(y_pred, targs)

        class_pred = torch.argmax(y_pred, dim=1)
        
        for i in class_pred:
            y_pred_lst.append(i.to("cpu").item())

        for i in targs:
            y_targ_lst.append(i.to("cpu").item())
    
    return y_targ_lst, y_pred_lst

In [ ]:
for epoch in range(1):
    train_loss = train(model, train_loader, optimizer, criterion)
    
    print(f"Epoch : {epoch} \t loss : {train_loss:.6f}")

y_targ, y_pred = eval(model, test_loader, criterion)
print(f"정확도 : {accuracy_score(y_targ, y_pred):.2f}%")